## class: Card(), Deck(), Hand(), Player(), Game()

    Card() --> Deck() --> Game() <-- Player() <-- Hand()

    
## global variable: STARTING_CHIPS


In [32]:
import sys,random

In [33]:
class Card(object):
    
    #用數字1-14代表字串A-K 13張牌
    number_to_name = {1:'A',11:'J',12:'Q',13:'K'}
    
    #靜態方法
    @staticmethod
    def HIDDEN_CARD():
        return [(' ' + ('_' * 9) + ' '),
                ('/' + (' ' * 9) + '\\'),
                ('|X' + (' ' * 8) + '|'),
                ('|         |'),
                ('|         |'),
                ('|         |'),
                ('|         |'),
                ('|         |'),
                ('|' + (' ' * 8) + 'X|'),
                ('\\' + ('_' * 9) +'/')]
    
    #suit代表花色
    def __init__(self, number, suit):
        self.number = number
        self.suit = suit
        self.name = (self.number_to_name[number] if number in self.number_to_name else str(number))
        self.value = min(number,10)
        self.display = []
        #給圖
        self.generate_display()
        
    def generate_display(self):
        #卡片上邊緣
        self.display.append(' ' + ('_' * 9) + ' ')
        self.display.append('/' + (' ' * 9) + '\\')
        self.display.append('|' + self.name + (' ' * (7 if self.name == '10' else 8)) + '|')
        #卡片花色形狀
        if self.suit == 'Spades':
            self.display.append('|' + (' ' * 4) + ',' + (' ' * 4) + '|')
            self.display.append('|   / \\   |')
            self.display.append('|  (_ _)  |')
            self.display.append('|   /_\\   |')
            self.display.append('|' + (' ' * 9) +'|')
        elif self.suit == 'Hearts':
            self.display.append('|   _ _   |')
            self.display.append('|  / ^ \\  |')
            self.display.append('|  \\   /  |')
            self.display.append('|   \\ /   |')
            self.display.append('|    `    |')
        elif self.suit == 'Clubs':
            self.display.append('|    _    |')
            self.display.append('|   (_)   |')
            self.display.append('|  (_)_)  |')
            self.display.append('|   /_\\   |')
            self.display.append('|' + (' ' * 9) +'|')
        elif self.suit == 'Diamonds':
            self.display.append('|' + (' ' * 9) +'|')
            self.display.append('|    /\\   |')
            self.display.append('|   <  >  |')
            self.display.append('|    \\/   |')
            self.display.append('|' + (' ' * 9) +'|')
        #卡片下邊緣
        self.display.append('|' + (' ' * (7 if self.name == '10' else 8)) + self.name + '|')
        self.display.append('\\' + ('_' * 9) +'/')


In [34]:
class Deck(object):
    unshuffled_deck = [
        Card(number, suit) for number in xrange(1,14) for suit in ['Spades','Hearts','Clubs','Diamonds']
    ]
    
    def __init__(self,number_of_deck =1):
        self.deck = self.unshuffled_deck * number_of_deck
        self.shuffle()
        
    def shuffle(self):
        return random.shuffle(self.deck)
    
    def deal_card(self):
        return self.deck.pop()
    
    def deal_hand(self):
        #餵Hand吃一卡片陣列，以計算出分數等屬性
        return Hand([self.deal_card(),self.deal_card()])

In [35]:
class Hand(object):
    def __init__(self,cards):
        self.hand = cards
    
    def all_scores(self):
        #因為數字1是代表ace，可是ace的value可以是1或11
        number_of_aces = sum(card.name == 'A' for card in self.hand)
        score = sum(card.value for card in self.hand)
        #看block8
        return [(score + (i * 10)) for i in xrange(number_of_aces + 1)]
    
    def possible_scores(self):
        #未爆掉的分數組合，如果為空陣列，代表真的爆了XD
        return [score for score in all_scores() if score <= 21]
    
    def add_card(self,card):
        self.hand.append(card)
    
    def print_hand(self, hide_first_card = False):
        for row in xrange(10):
            for idx, card in enumerate(self.hand):
                if hide_first_card and idx == 0:
                    sys.stdout.write(Card.HIDDEN_CARD[row])
                else:
                    sys.stdout.write(card.display[row])
                sys.stdout.write('  ') 
            print('')
        print('')

In [ ]:
STARTING_CHIPS = 100
class Player(object):
    def __init__(self,name='YOU', chips=STARTING_CHIPS):
        self.name = name
        self.chips = chips
        self.bet = 0
        self.hand = None
        self.stand = False
        self.wins = 0
        self.pushes = 0
        self.losses = 0
    
    def reset(self):
        self.bet = 0
        self.hand = None
        self.stand = False
        
    def win(self, winnings):
        print 'You win %d chip(s).' % winnings
        self.chips += winnings
        self.wins += 1
    
    def lose(self, loss):
        print 'You loss %d chip(s).' % loss
        self.chips -= loss
        self.losses += 1
        
    def push(self, bet):
        print 'You keep %d chip(s).' % bet
        self.pushes += 1
        
    def hit(self):
        self.hand.add_card(self)
    
    def is_bust(self):
        return len(self.hand.possible_scores()) == 0
    
    def scores(self):
        #如果爆掉就回傳all_scores list
        return self.hand.all_scores() if self.is_bust() else self.hand.possible_scores()
    
    def max_score(self):
        return max(self.scores())
    
    def min_score(self):
        return min(self.scores())      

In [ ]:
class Game(object):
    def __init__(self):
        self.dealer = Player(name='Dealer')
        self.player = Player(name='player')
        self.deck = Deck()
        
    def print_talble(self, hide_dealer_card = True):
        print '===============\n'
        #如果最大數爆了就取最小
        print 'Dealer: %s' % ('' if hide_dealer_card else (
            str(max(self.dealer.max_score())) if str(max(self.dealer.max_score())) <= 21 else str(max(self.dealer.min_score()))))
        self.dealer.hand.print_hand(hide_first_card = hide_dealer_card)
        print '%s: %d' % (self.player.name, self.player.max_score() if self.player.max_score() <= 21 else self.player.min_score())  
        self.player.hand.print_hand()
                              
    def set_up(self):
        self.deck = Deck()
        self.dealer.reset()
        self.player.reset()
        print '\nYou have:'
        print 'Won    %d games' % self.player.wins
        print 'Pushed %d games' % self.player.pushes
        print 'Lost   %d games\n' % self.player.losses
        print 'You have %d chips remaining' % self.player.chips
    
    def get_player_bet(self):
        while True:       
            bet_input = raw_input("Enter bet for this hand(or 'exit' to quit): ").strip().lower()
            if bet_input == 'exit':
                print 'Thank you for playing :)'
                sys.exit(0) 
            if not bet_input.isdigit():
                print 'Input Error! Please try again.'
            elif int(bet_input) > self.player.chips:
                print 'You do not have enough chips for that bet! Please try again.'
            elif int(bet_input) <= 0:
                print "You can't bet less than 1 chip! Please try again."
            else:
                return int(bet_input)
            
    def deal_initial_hands(self):
        self.player.hand = self.deck.deal_hand()
        self.dealer.hand = self.deck.deal_hand()
        
    def blackjack_check(self):
        if self.player.max_score() == 21 and self.dealer.max_score() == 21:
            self.print_table(hide_dealer_card=False)
            print 'You PUSH!'
            self.player.push(self.player.bet)
            return True
        elif self.player.max_score() == 21:
            self.print_table(hide_dealer_card=False)
            print 'BLACKJACK!'
            self.player.win(self.player.bet * 1.5)
            return True
        elif self.dealer.max_score() == 21:
            self.print_table(hide_dealer_card=False)
            print 'Dealer BLACKJACK!'
            self.player.lose(self.player.bet)
            return True
        self.print_table()
        return False
    def player_choices(self):
        print ''
        while not self.player.stand and not self.player.is_bust():
            correct_input = False
            while not correct_input:
                hit_or_stand = raw_input("Enter 'hit' or 'stand': ").strip().lower()
                if (hit_or_stand != 'hit' and hit_or_stand != 'stand' and
                    hit_or_stand != 'h' and hit_or_stand != 's'):
                    print 'Input Error! Please try again.'
                else:
                    correct_input = True
            if hit_or_stand == 'hit' or hit_or_stand == 'h':
                # Hit and re-print table
                self.player.hit(self.deck.deal_card())
                self.print_table()
            elif hit_or_stand == 'stand' or hit_or_stand == 's':
                self.player.stand = True

    def check_player_bust(self):
        if self.player.is_bust():
            self.player.stand = True
            print 'You BUST!'
            self.player.lose(self.player.bet)

    def dealer_choices(self):
        if not self.player.is_bust():
            self.print_table(hide_dealer_card=False)
            while self.dealer.max_score() < 17:
                self.dealer.hit(self.deck.deal_card())
                self.print_table(hide_dealer_card=False)

    def final_outcome(self):
        if not self.player.is_bust():
            if self.dealer.is_bust():
                print 'Dealer BUST!'
                self.player.win(self.player.bet)
            elif self.player.max_score() == self.dealer.max_score():
                print 'You PUSH!'
                self.player.push(self.player.bet)
            elif self.player.max_score() > self.dealer.max_score():
                print 'You WIN!'
                self.player.win(self.player.bet)
            elif self.player.max_score() < self.dealer.max_score():
                print 'You LOSE!'
                self.player.lose(self.player.bet)

    def play(self):
        print 'Welcome to Blackjack!'

        while self.player.chips > 0:
            # 1: Set up
            self.set_up()
            
            # 2: Get player's bet
            self.player.bet = self.get_player_bet()

            self.deal_initial_hands()

            # 3: Check for blackjacks and print initial table
            if self.blackjack_check():
                print '===================================='
                continue

            # 4: Ask user hit or stand until stand/bust
            self.player_choices()

            # 5: print display again
            self.print_table()

            # 6: Check if player bust
            self.check_player_bust()

            # 7: Once stand, reveal dealer card, dealer logic
            self.dealer_choices()

            # 8: Check for final outcome
            self.final_outcome()

            print '===================================='

        print '%s are out of money!' % self.player.name
        print 'Game over :('

if __name__ == '__main__':
    game = Game()
    game.play()                   

Welcome to Blackjack!

You have:
Won    0 games
Pushed 0 games
Lost   0 games

You have 100 chips remaining


## note

1. The @ symbol is used for class, function and method decorators.

## block1: print a card

In [ ]:
HIDDEN_CARD = [(' ' + ('_' * 9) + ' '),
                ('/' + (' ' * 9) + '\\'),
                ('|X' + (' ' * 8) + '|'),
                ('|         |'),
                ('|         |'),
                ('|         |'),
                ('|         |'),
                ('|         |'),
                ('|' + (' ' * 8) + 'X|'),
                ('\\' + ('_' * 9) +'/')]

import sys
for row in xrange(10):
    sys.stdout.write(HIDDEN_CARD[row])
    print('') #for next line

## block2: sys.stdout vs print

    1. in python 2.6+, print is a statement, see:http://www.takka.com.hk/jstutor/ch6/ch6.htm
    2. in python 3, print is a function
    3. print formats the inputs (space between args and newline at the end) and calls the write function of sys.stdout

## block3: key in dict

In [ ]:
dic = {1:'A',11:'J',12:'Q',13:'K'}
1 in dic

## block4: if statement in one line

In [ ]:
v = (x if ... else ...)

## block5: min()

In [ ]:
v = min(x,10)

## block6: simplify for loop 

In [ ]:
def add_num(x,y):
    return x+y

print [add_num(x, y) for x in xrange(1,5) for y in [1.1,2.2,3.3]]

## block7: get specific item count from list

In [ ]:
lst = [1,2,3,4,5,6,7,8,9,10]
print sum(i % 2 == 0 for i in lst)

## block8: for loop again

In [ ]:
# you have 2 aces and one 3 in hand, and your possible score is:
min_score = 1 + 1 + 3
print [ min_score + (i * 10) for i in xrange(2+1)] 

## block9: enumerate

In [ ]:
class Dog(object):
    def __init__(self,breed,age):
        self.breed = breed
        self.age = age
        
dogs = [Dog(breed,age) for breed in ['A','B'] for age in xrange(1,3)]

for x,y in enumerate(dogs):
    print x
    print y
    print '------'

## block10: how to quit

In [ ]:
sys.exit(0)

## block11: isdigit()

In [ ]:
x = '1'
x.isdigit()

## block12 :__main__ and __name__

When the Python interpreter reads a source file, it executes all of the code found in it.

Before executing the code, it will define a few special variables. For example, if the python interpreter is running that module (the source file) as the main program, it sets the special __name__ variable to have a value "__main__". If this file is being imported from another module, __name__ will be set to the module's name.

on the command line. After setting up the special variables, it will execute the import statement and load those modules. It will then evaluate the def block, creating a function object and creating a variable called myfunction that points to the function object. It will then read the if statement and see that __name__ does equal "__main__", so it will execute the block shown there.

One of the reasons for doing this is that sometimes you write a module (a .py file) where it can be executed directly. Alternatively, it can also be imported and used in another module.

By doing the main check, you can have that code only execute when you want to run the module as a program and not have it execute when someone just wants to import your module and call your functions themselves.

### in A_Module.py run:
    
    print '__name__: %s' % __name__ 
    
    => it prints __main__
    
### in B_Module.py import A_Module.py, and run A_Module.py's code:

    print '__name__: %s' % __name__ 
    
    => it prints A_Module   
